Implementing Amitabha-Bose Model for Phase Precession

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

In [ ]:
#setting parameters

#Reversal Potentials

v_ca = 120.0
v_k = -84.0
v_l = -60.0

#capacitnance
c_m = 20.0

#max. conductances
g_ca = 4.4
g_k = 8.0
g_l = 2.0

#applied currents
i_i = 120.0
i_p = 105.0
i_t = 92.0

#some epsilon
eps = 0.005

#half-activation voltages for gating functions
v_1p = -1.2
v_1t = -1.2
v_3p = 2.0
v_3i = -25.0

#activation-inactivation sensitivities for gating functions
v_2p = 18.0
v_2i = 18.0
v_1i = 18.0
v_4p = 30.0
v_4i = 10.0

#half-activation and senstivity for synaptic gate
v_5 = 0.0
v_6 = 10.0

#synapse rise-decay rates
alpha = 2.0
beta = 0.5

#reversal potentials of synaptic currents
v_pi = 80.0
v_ip = -80.0
v_ti = -80.0

#max. conductances of synaptic currents
g_pi = 1.0
g_ip = 1.0
g_ti = 1.0











In [ ]:
#helper functions

def m_inf(v,v1,v2): #steady state actviation function
    return 0.5 * (1 + np.tanh((v - v1) / v2))

def w_inf(v,v3,v4): #steady state inactivation function
    return 0.5 * (1 + np.tanh((v - v3) / v4))

def tau_w(v,v3,v4): #time constant for inactivation variable
    return (eps * np.cosh((v - v3) / (2 * v4)))

def sigmoid_synv(v,v5,v6): #synaptic gate sigmoid function
    return 0.5 * (1 + np.tanh((v - v5) / v6))

In [ ]:
#differential equations for the system

def dVp_dt(Vp,i_p,s_ip):

    #calcium current
    i_ca = g_ca * m_inf(Vp,v_1p,v_2p)*(Vp - v_ca)
    #potassium current
    i_k = g_k * w_inf(Vp,v_3p,v_4p)*(Vp - v_k)
    #leak current
    i_l = g_l * (Vp - v_l)
    #synaptic current from inhibitory interneuron
    i_syn = g_ip * s_ip * (Vp - v_ip)
    #total current
    i_total = i_ca + i_k + i_l + i_syn - i_p
    #dv/dt
    return -i_total / c_m

def dVi_dt(Vi,i_i,s_pi,s_ti):

    #calcium current
    i_ca = g_ca * m_inf(Vi,v_1i,v_2i)*(Vi - v_ca)
    #potassium current
    i_k = g_k * w_inf(Vi,v_3i,v_4i)*(Vi - v_k)
    #leak current
    i_l = g_l * (Vi - v_l)
    #synaptic current from pyramidal cell
    i_syn_pi = g_pi * s_pi * (Vi - v_pi)
    #synaptic current from thalamic input
    i_syn_ti = g_ti * s_ti * (Vi - v_ti)
    #total current
    i_total = i_ca + i_k + i_l + i_syn_pi + i_syn_ti - i_i
    #dv/dt
    return -i_total / c_m

def dVT_dt(Vt,i_t):

    #calcium current
    i_ca = g_ca * m_inf(Vt,v_1t,v_2p)*(Vt - v_ca)
    #potassium current
    i_k = g_k * w_inf(Vt,v_3p,v_4p)*(Vt - v_k)
    #leak current
    i_l = g_l * (Vt - v_l)
    #total current
    i_total = i_ca + i_k + i_l - i_t
    #dv/dt
    return -i_total / c_m

def dwp_dt(Vp):